In [ ]:
import sys
sys.path.append('../src')
from evaluation import evaluate_model, print_metrics
import numpy as np
from data_preprocessing import load_data, prepare_data
from model import LSTMModel
import torch
from torch.utils.data import TensorDataset, DataLoader
import pickle
import json

In [3]:
df = load_data('../data/household_power_consumption.txt')
(X_train, y_train), (X_val, y_val), (X_test, y_test), scaler = prepare_data(df)

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

c:\Users\MSI 1\Documents\Energy-Consumptiom\notebooks\../src\data_preprocessing.py:7: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(filepath,
c:\Users\MSI 1\Documents\Energy-Consumptiom\notebooks\../src\data_preprocessing.py:7: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(filepath,


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMModel(input_size = 1, hidden_size = 50, num_layers = 1, output_size = 1)
model.load_state_dict(torch.load('../models/lstm_energy_model.pth', map_location=device))
model.eval()

model = model.to(device)


def get_predictions_batched(model, data_tensor, device, batch_size=1024):
    """Get predictions in batches to avoid OOM errors"""
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i:i+batch_size].to(device)
            batch_pred = model(batch).cpu().numpy()
            predictions.append(batch_pred)
    
    return np.concatenate(predictions).flatten()

print("Getting test predictions...")
test_predictions = get_predictions_batched(model, X_test_tensor, device)

print("Getting train predictions...")
train_predictions = get_predictions_batched(model, X_train_tensor, device)

print("Getting validation predictions...")
val_predictions = get_predictions_batched(model, X_val_tensor, device)

print("\n=== TEST SET METRICS ===")
test_metrics = evaluate_model(test_predictions, y_test, scaler)
print_metrics(test_metrics)

print("\n=== VALIDATION SET METRICS ===")
val_metrics = evaluate_model(val_predictions, y_val, scaler)
print_metrics(val_metrics)

print("\n=== TRAIN SET METRICS ===")
train_metrics = evaluate_model(train_predictions, y_train, scaler)
print_metrics(train_metrics)

Getting test predictions...
Getting train predictions...
Getting validation predictions...

=== TEST SET METRICS ===

MODEL EVALUATION METRICS
MAE (Mean Absolute Error):      0.0804 kW
RMSE (Root Mean Squared Error): 0.2042 kW
MAPE (Mean Abs Percentage Err): 9.76%
R² Score:                       0.9439

Directional Accuracy:           44.51%
Peak Detection Precision:       87.76%
Peak Detection Recall:          90.38%


=== VALIDATION SET METRICS ===

MODEL EVALUATION METRICS
MAE (Mean Absolute Error):      0.0914 kW
RMSE (Root Mean Squared Error): 0.2319 kW
MAPE (Mean Abs Percentage Err): 7.93%
R² Score:                       0.9517

Directional Accuracy:           45.29%
Peak Detection Precision:       87.90%
Peak Detection Recall:          89.62%


=== TRAIN SET METRICS ===

MODEL EVALUATION METRICS
MAE (Mean Absolute Error):      0.0918 kW
RMSE (Root Mean Squared Error): 0.2443 kW
MAPE (Mean Abs Percentage Err): 9.75%
R² Score:                       0.9498

Directional Accuracy:   

: 